# BERT 문장 유사도

In [1]:
from transformers import TFBertForNextSentencePrediction, AutoTokenizer

c:\Users\qq\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL_NAME = 'bert-base-uncased'
model = TFBertForNextSentencePrediction.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

All PyTorch model weights were used when initializing TFBertForNextSentencePrediction.

All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.
c:\Users\qq\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


인덱스 0 : 실제 다음 문장으로 올 수 있음

인덱스 1 : 서로 상관없는 문장

In [3]:
# prompt와 next_sentence를 연관있는 문장으로 작성
prompt = 'In korea, pizza is delicious.'
next_sentence = 'pizza is eaten with the use of knife and fork.'

In [4]:
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

# 인코딩된 전체 내역 확인
encoding

{'input_ids': <tf.Tensor: shape=(1, 21), dtype=int32, numpy=
array([[  101,  1999,  4420,  1010, 10733,  2003, 12090,  1012,   102,
        10733,  2003,  8828,  2007,  1996,  2224,  1997,  5442,  1998,
         9292,  1012,   102]])>, 'token_type_ids': <tf.Tensor: shape=(1, 21), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])>, 'attention_mask': <tf.Tensor: shape=(1, 21), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])>}

In [5]:
# input_ids 확인
print(encoding['input_ids'])

tf.Tensor(
[[  101  1999  4420  1010 10733  2003 12090  1012   102 10733  2003  8828
   2007  1996  2224  1997  5442  1998  9292  1012   102]], shape=(1, 21), dtype=int32)


In [6]:
# input_ids[0] 정보를 가지고 decode 실행해보기
print(tokenizer.decode(encoding['input_ids'][0]))

[CLS] in korea, pizza is delicious. [SEP] pizza is eaten with the use of knife and fork. [SEP]


In [7]:
# token_type_ids 확인 (0:prompt, 1:next_sentence)
encoding['token_type_ids']

<tf.Tensor: shape=(1, 21), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])>

In [8]:
# input_ids로 인코딩, 2개의 문장을 입력했으므로, token_type_ids로 명시
logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]
logits

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 6.331632 , -6.2720737]], dtype=float32)>

In [9]:
import tensorflow as tf

# logits을 softmax함수를 통해 확률값으로 바꿔주어야 함
softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print(probs)

tf.Tensor([[9.9999666e-01 3.3595293e-06]], shape=(1, 2), dtype=float32)


In [10]:
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [0]


In [15]:
prompt = 'In korea, pizza is delicious'
next_sentence = 'the blue sky.'
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print(probs)

tf.Tensor([[0.00397046 0.9960296 ]], shape=(1, 2), dtype=float32)


In [16]:
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [1]
